In [4]:
import pandas as pd
import os, sys, torch
from torchvision.transforms.functional import to_pil_image
from PIL import Image

sys.path.append(os.path.abspath('../../modules'))
sys.path.append(os.path.abspath('../../modules/celeba_male'))
import datapipe as dp
import utility as ut
from vae import VAE
import classifier as cl
from torchsummary import summary
import vae_loss as vl
import vae_viz as viz
import vae_train as vt
import vae_ortho as vo
import vae_surgery as vs
import vae_os as vos
import vae_ascent as va
import vae_ad as vad

device = ut.get_device()


data_folder = '../../data/CelebA/dataset'
model_folder = '../../data/CelebA/vae'
data = dp.CelebAData().get_dataloader(batch_size=128, shuffle=False, drop_last=True)

model = VAE()
model = torch.load(f'{model_folder}/vae_200.pth', weights_only=False, map_location=device)
model.to(device)
model.eval()

output_dir = '../../data/CelebA/dataset-reconstructed/img_align_celeba_reconstructed'
os.makedirs(output_dir, exist_ok=True)

# image_counter = 1
# for i, (img_tensor, _) in enumerate(data):
#     img_tensor = img_tensor.to(device)
#     with torch.no_grad():
#         img_tensor, _, _ = model(img_tensor)
#     for img in img_tensor:
#         pil_img = to_pil_image(img.cpu())
#         filename = f"{image_counter:06d}.jpg"
#         pil_img.save(os.path.join(output_dir, filename), format='JPEG', quality=100)
#         image_counter += 1
#         print(f"Working on image #{image_counter}", end='\r')
print(ut.get_file_count(output_dir))

202599


In [7]:
import numpy as np
data = dp.CelebAData().get_dataloader(batch_size=128, shuffle=False, drop_last=True)
all_feat = []
for i, (img_tensor, _) in enumerate(data):
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        feat = model.encoder(img_tensor)

    all_feat.append(feat.cpu())

    print(f"Processed batch {i + 1}", end='\r')

# Concatenate all batches
all_feat = torch.cat(all_feat, dim=0).numpy()


# Save as .npy files
output_dir = "../../data/CelebA/vae"
# os.makedirs(output_dir, exist_ok=True)
np.save(os.path.join(output_dir, "features_200.npy"), all_feat)
print(f"\nSaved {all_feat.shape[0]} mu/logvar vectors to {output_dir}/")

Processed batch 1582
Saved 202496 mu/logvar vectors to ../../data/CelebA/vae/


In [6]:
all_feat.shape

(202496, 512, 2, 2)